#### Names of people in the group

Please write the names of the people in your group in the next cell.

Joachim Maksim

Jørgen Katralen

In [0]:
# Deleting tables left from previous runs in case they still exist after deleting an inactive cluster
dbutils.fs.rm("/user", recurse=True)

Out[27]: True

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-31e318ea-344d-49a0-be2c-8e2850ca45e8/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading PySpark modules that we need
import unittest
from collections import Counter
from pyspark.sql import DataFrame
from pyspark.sql.types import *

#### Subtask 1: defining the schema for the data
Typically, the first thing to do before loading the data into a Spark cluster is to define the schema for the data. Look at the schema for 'badges' and try to define the schema for other tables similarly.

In [0]:
# Defining a schema for 'badges' table
badges_schema = StructType([StructField('UserId', IntegerType(), False),
                            StructField('Name', StringType(), False),
                            StructField('Date', TimestampType(), False),
                            StructField('Class', IntegerType(), False)])

# Defining a schema for 'posts' table
posts_schema = StructType([StructField('Id', IntegerType(), False),
                            StructField('ParentId', IntegerType(), True),
                            StructField('PostTypeId', IntegerType(), False),
                            StructField('CreationDate', TimestampType(), False),
                            StructField('Score', IntegerType(), False),
                            StructField('ViewCount', IntegerType(), False),
                            StructField('Body', StringType(), False),
                            StructField('OwnerUserId', IntegerType(), False),
                            StructField('LastActivityDate', TimestampType(), False),
                            StructField('Title', StringType(), True),
                            StructField('Tags', StringType(), True),
                            StructField('AnswerCount', IntegerType(), False),
                            StructField('CommentCount', IntegerType(), False),
                            StructField('FavoriteCount', IntegerType(), False),
                            StructField('ClosedDate', TimestampType(), True)])
## YOUR IMPLEMENTATION ##

# Defining a schema for 'users' table
users_schema = StructType([StructField('Id', IntegerType(), False),
                            StructField('Reputation', IntegerType(), False),
                            StructField('CreationDate', TimestampType(), False),
                            StructField('DisplayName', StringType(), False),
                            StructField('LastAccessDate', TimestampType(), False),
                            StructField('AboutMe', StringType(), True),
                            StructField('Views', IntegerType(), False),
                            StructField('UpVotes', IntegerType(), False),
                            StructField('DownVotes', IntegerType(), False)])
## YOUR IMPLEMENTATION ##

# Defining a schema for 'comments' table
comments_schema = StructType([StructField('PostId', IntegerType(), False),
                            StructField('Score', IntegerType(), False),
                            StructField('Text', StringType(), False),
                            StructField('CreationDate', TimestampType(), False),
                            StructField('UserId', IntegerType(), False)])
## YOUR IMPLEMENTATION ##

#### Subtask 2: implementing two helper functions
Next, we need to implement two helper functions:
1. 'load_csv' that as input argument receives path for a CSV file and a schema and loads the CSV pointed by the path into a Spark DataFrame and returns the DataFrame;
2. 'save_df' receives a Spark DataFrame and saves it as a Parquet file on DBFS.

Note that the column separator in CSV files is TAB character ('\t') and the first row includes the name of the columns. 

BTW, DBFS is the name of the distributed filesystem used by Databricks Community Edition to store and access data.

In [0]:
def load_csv(source_file: "path for the CSV file to load", schema: "schema for the CSV file being loaded as a DataFrame") -> DataFrame:
    return spark.read.format("csv").option("delimiter", "\t").option("header", True).schema(schema).load(source_file)

def save_df(df: "DataFrame to be saved", table_name: "name under which the DataFrame will be saved") -> None:
    df.write.format("parquet").save(f"/user/hive/warehouse/{table_name}") # path was not specified, discovered in task 2
    ## YOUR IMPLEMENTATION ##

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully. Moreover, at the end there should be four Parquet files named 'badges', 'comments', 'posts', and 'users' in '/user/hive/warehouse'.

Note that we assumed that the data for the project has already been stored on DBFS on the '/FileStore/tables/' path. (I mean as 'badges_csv.gz', 'comments_csv.gz', 'posts_csv.gz', and 'users_csv.gz'.)

In [0]:
%%unittest_main
class TestTask1(unittest.TestCase):
   
    # test 1
    def test_load_badges(self):
        result = load_csv(source_file="/FileStore/tables/badges_csv.gz", schema=badges_schema)
        self.assertIsNotNone(result, "Badges dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 105640, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['UserId', 'Name', 'Date', 'Class']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 2
    def test_load_posts(self):
        result = load_csv(source_file="/FileStore/tables/posts_csv.gz", schema=posts_schema)
        self.assertIsNotNone(result, "Posts dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 61432, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'ParentId', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
                                    'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
                                    'ClosedDate']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 3
    def test_load_comments(self):
        result = load_csv(source_file="/FileStore/tables/comments_csv.gz", schema=comments_schema)
        self.assertIsNotNone(result, "Comments dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 58735, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['PostId', 'Score', 'Text', 'CreationDate', 'UserId']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 4
    def test_load_users(self):
        result = load_csv(source_file="/FileStore/tables/users_csv.gz", schema=users_schema)
        self.assertIsNotNone(result, "Users dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 91616, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'AboutMe',
                                    'Views', 'UpVotes', 'DownVotes']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    # test 5
    def test_save_dfs(self):
        """ This test doesn't actually assert anything, so it only fails if the code fails to execute... """
        dfs = [("/FileStore/tables/users_csv.gz", users_schema, "users"),
               ("/FileStore/tables/badges_csv.gz", badges_schema, "badges"),
               ("/FileStore/tables/comments_csv.gz", comments_schema, "comments"),
               ("/FileStore/tables/posts_csv.gz", posts_schema, "posts")
               ]

        for i in dfs:
            df = load_csv(source_file=i[0], schema=i[1])
            save_df(df, i[2])

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 22.090s

OK
Out[29]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'Spark Application', 'SparkSession', 'Transformations', 'Action', and 'Lazy Evaluation' mean in the context of Spark?

Write your descriptions in the next cell.

##### Spark Application
The spark application is the application that we run to generate results through our code. It takes care of the Driver and Worker processes. These worker processes execute the code we write and takes care of the computations, the driver keeps track of and organizes them. 

##### SparkSession
A  spark session is the entry point of a Spark application. You can look at it like an interface for the Spark application. It wraps up a lot of functionality (like hive, SQL and Spark) for ease of use.

##### Transformations
Transformations (does as the name suggest) transform/changes the data you have through the code. It's a function that produces new RDD from existing RDDs. Each time it creates new RDD we apply a transformation. An RDD is one of the fundamental data strructures we can use in Spark. They are collections of objects that are fault-tolerant 


##### Action
Actions in Spark RDD is the operations that return raw values. Everything that doesn't return an RDD is considered an action in Spark.

##### Lazy Evaluation
This is a concept in Spark that makes it so you can apply an indefinite number of transformations to the data, but Spark won't execute them before an action is called. Thus it can save some time or make computations more efficient by freeing up processing power until it is needed.